In [ ]:
%pip install pandas numpy matplotlib seaborn plotly scipy scikit-learn unidecode statsmodels

Note: you may need to restart the kernel to use updated packages.


# Fonte dos dados: [Chegadas de Turistas Internacionais](https://dados.turismo.gov.br/dataset/chegada-de-turistas-internacionais)

In [20]:
import pandas as pd
from glob import glob
from unidecode import unidecode
import plotly.express as px
import plotly.graph_objects as go
from prophet import Prophet

import warnings
warnings.filterwarnings('ignore')

In [3]:
# to run on colab
#from google.colab import drive
#drive.mount('/content/drive')

In [5]:
#data_folder = 'drive/MyDrive/Unifor/G707_Unifor/dados/' # to run on colab
data_folder = './data/'

# Preparação dos dados

In [4]:
# inicializa dataframe
df = pd.DataFrame([])

# lê todos os CSVs na pasta dados
files = glob(f'{data_folder}chegadas*.csv')

if len(files) == 0:
    print("Nenhum arquivo encontrado.")

else:
  for f in files:
      print(f)
      df_ano = pd.read_csv(f, encoding="ISO-8859-1", delimiter=';', on_bad_lines='skip')
      # processa nomes das colunas
      col_names = pd.Series(df_ano.columns).apply(lambda x: unidecode(x).lower().replace(' ', '_').replace('ordem', 'cod')).values
      df_ano.columns = col_names
      # seleciona estado do Ceará e dados com chegadas maior que 0
      df_ano = df_ano.query("cod_uf == 6 and chegadas > 0")
      df = pd.concat([df, df_ano])

  df.dropna(axis=1, inplace=True)
  df.loc[:,df.select_dtypes('object').columns] = df.select_dtypes('object').map(unidecode)

  group_cols = ['continente', 'pais', 'ano', 'mes']
  df = df[group_cols + ['chegadas']].groupby(group_cols, as_index=False).sum()

  # padroniza nome do mes
  df['mes'] = df['mes'].str.lower()

  df.to_csv(f'{data_folder}dados_processados.csv', index=False)

./dados/chegadas_2024.csv
./dados/chegadas_2018.csv
./dados/chegadas_2019.csv
./dados/chegadas_2021.csv
./dados/chegadas_2020.csv
./dados/chegadas_2022.csv
./dados/chegadas_2023.csv
./dados/chegadas_2015.csv
./dados/chegadas_2017.csv
./dados/chegadas_2016.csv


In [5]:
df = pd.read_csv(f'{data_folder}dados_processados.csv')

## Descrição do Dataset

O dataset contém registros de chegadas de turistas no estado do Ceará de 2025 a 2024. Cada registro apresenta as seguintes colunas:

**continente**: Indica a região geográfica de onde veio o turista (ex.: Europa, Ásia, África, etc.).

**pais**: Nome do país de onde veio o turista.

**ano**: Ano em que os dados foram registrados.

**mes**: Mês da visita (em formato textual).

**chegadas**: Número de chegadas de turistas registrados no período.

Esses dados permitem identificar tendências temporais (por ano ou por mês) e padrões geográficos, possibilitando análises sobre o fluxo de chegadas em diferentes regiões.

# Análise Exploratória dos Dados (EDA)

In [6]:
# Cálculo das estatísticas básicas para a coluna 'chegadas'
mean_chegadas = df['chegadas'].mean()
median_chegadas = df['chegadas'].median()
std_chegadas = df['chegadas'].std()

print("\nEstatísticas de chegadas:")
print(f"\tMédia: {mean_chegadas:.0f}")
print(f"\tMediana: {median_chegadas:.0f}")
print(f"\tDesvio Padrão: {std_chegadas:.0f}\n\n")

# Conversão dos nomes dos meses para números (tratando letras minúsculas)
df['mes_lower'] = df['mes'].str.lower()
meses = {"janeiro":1, "fevereiro":2, "marco":3, "março":3, "abril":4, "maio":5, "junho":6,
         "julho":7, "agosto":8, "setembro":9, "outubro":10, "novembro":11, "dezembro":12}
df['mes_num'] = df['mes_lower'].map(meses)

# 1. Gráfico de histórico geral
df['data'] = pd.to_datetime(df['ano'].astype(str) + '-' + df['mes_num'].astype(str) + '-01')
# Agrupando os dados por data para obter o total de chegadas por mês
df_data = df.groupby('data')['chegadas'].sum().reset_index()
# Criando o gráfico de linha com todo o histórico de meses
fig = px.line(df_data, x='data', y='chegadas', markers=True, title="Histórico de Chegadas por Mês em Todos os Anos")
fig.update_xaxes(title="Data")
fig.update_yaxes(title="Total de Chegadas")
fig.show()

# 2. Gráfico de linha: Total de chegadas por ano
df_ano = df.groupby('ano')['chegadas'].sum().reset_index()
fig1 = px.line(df_ano, x='ano', y='chegadas', markers=True,
               title="Total de Chegadas por Ano")
fig1.show()

# 3. Gráfico de barras: Total de chegadas por continente
df_continente = df.groupby('continente')['chegadas'].sum().reset_index()
fig2 = px.bar(df_continente, x='continente', y='chegadas',
              title="Total de Chegadas por Continente")
fig2.show()

# 4. Gráfico de barras: Top 10 países com mais chegadas
df_pais = df.groupby('pais')['chegadas'].sum().reset_index().sort_values(by='chegadas', ascending=False)
top10 = df_pais.head(10)
fig3 = px.bar(top10, x='pais', y='chegadas',
              title="Top 10 Países")
fig3.show()

# 5. Gráfico de linha: Chegadas por mês
df_mes = df.groupby('mes_num')['chegadas'].sum().reset_index()
# Criação de um dicionário para mapear números para nomes dos meses
month_mapping = {
    1: "Janeiro", 2: "Fevereiro", 3: "Março", 4: "Abril",
    5: "Maio", 6: "Junho", 7: "Julho", 8: "Agosto",
    9: "Setembro", 10: "Outubro", 11: "Novembro", 12: "Dezembro"
}
# Mapeamento dos números para nomes dos meses
df_mes["mes_nome"] = df_mes["mes_num"].map(month_mapping)
fig4 = px.bar(df_mes, x='mes_nome', y='chegadas',
               title="Chegadas por Mês")
fig4.update_xaxes(title="Mês", tickmode="linear")
# Adiciona uma linha de tendência ao gráfico
fig4.add_trace(go.Scatter(x=df_mes["mes_nome"], y=df_mes["chegadas"],
                          mode="lines+markers", name="Tendência"))
fig4.show()


Estatísticas de chegadas:
	Média: 110
	Mediana: 7
	Desvio Padrão: 293




# Predição de chegadas

In [104]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Prepare the data
monthly_data = df.groupby(['ano', 'mes_num'])['chegadas'].sum().reset_index()
data = monthly_data['chegadas'].values

# Initialize and train Exponential Smoothing model
# Using seasonal_periods=12 for monthly data, but with simpler model parameters
model = ExponentialSmoothing(
    data,
    seasonal_periods=12,
    trend='add',  # Remove trend component
    seasonal='add',  # Remove seasonal component since we don't have enough cycles
    use_boxcox=True
)
fitted_model = model.fit()

# Make predictions for next 12 months
forecast = fitted_model.forecast(12)

# Create DataFrame with predictions
# Ensure predictions are non-negative by using maximum of prediction and 0
pred_df = pd.DataFrame({
    'mes_num': range(1, 13),
    'chegadas': np.maximum(forecast, 0).astype(int)
})

In [117]:
# 1. Add a 'data' column to the historical data (df)
df['data'] = pd.to_datetime(df['ano'].astype(str) + '-' + df['mes_num'].astype(str) + '-01')

# 2. Add a 'data' column to the prediction data (pred_df) with year 2025
pred_df['data'] = pd.to_datetime('2025-' + pred_df['mes_num'].astype(str) + '-01')

# 3. Select the desired columns for both DataFrames
historical_data = df[['data', 'chegadas']]
historical_data['type'] = 'historical'
prediction_data = pred_df[['data', 'chegadas']]
prediction_data['type'] = 'prediction'

# 4. Concatenate the two DataFrames
combined_df = pd.concat([historical_data, prediction_data])

# 5. Sort by 'data' for better visualization
combined_df.sort_values(by=['data'], inplace=True)

In [135]:
# Criar gráfico de linhas com Plotly
import plotly.express as px

# Agrupar os dados somando as chegadas por data e tipo
grouped_df = combined_df.groupby(['data', 'type'])['chegadas'].sum().reset_index()

# Filtrar dados a partir de 2023
grouped_df = grouped_df[grouped_df['data'].dt.year >= 2023]

fig = px.line(grouped_df, 
              x='data', 
              y='chegadas',
              markers=True,
              color='type',
              color_discrete_map={'historical': 'blue', 'prediction': 'orange'},
              title='Total de Chegadas: Histórico e Previsões',
              labels={
                  'data': 'Data',
                  'chegadas': 'Número de Chegadas',
                  'type': 'Tipo'
              })

# Atualizar layout
fig.update_layout(
    xaxis_title='Data',
    yaxis_title='Número de Chegadas',
    hovermode='x unified',
    #width=1500,
    xaxis=dict(
        tickformat='%m/%Y',  # Format as MM/YYYY
        tickmode='array',
        ticktext=grouped_df['data'].dt.strftime('%m/%Y').unique(),
        tickvals=grouped_df['data'].unique()
    )
)

# Traduzir legendas do tipo de linha
fig.for_each_trace(lambda t: t.update(name=t.name.replace('historical', 'histórico').replace('prediction', 'previsão')))

fig.show()